# PDBe API Training

### PDBe search

Searching with a sequence

In [1]:
from pprint import pprint
from api_modules import run_sequence_search, pandas_dataset, pandas_count, pandas_plot, pandas_plot_multi_groupby

We will search for a sequence with an example sequence from UniProt P24941 -
Cyclin-dependent kinase 2

In [2]:
sequence_to_search = """
MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELNH
PNIVKLLDVIHTENKLYLVFEFLHQDLKKFMDASALTGIPLPLIKSYLFQLLQGLAFCHS
HRVLHRDLKPQNLLINTEGAIKLADFGLARAFGVPVRTYTHEVVTLWYRAPEILLGCKYY
STAVDIWSLGCIFAEMVTRRALFPGDSEIDQLFRIFRTLGTPDEVVWPGVTSMPDYKPSF
PKWARQDFSKVVPPLDEDGRSLLSQMLHYDPNKRISAKAALAHPFFQDVTKPVPHLRL"""

filter_list = ['pfam_accession', 'pdb_id', 'molecule_name', 'ec_number',
               'uniprot_accession_best', 'tax_id']

first_results = run_sequence_search(sequence_to_search, filter_terms=filter_list)

https://www.ebi.ac.uk/pdbe/search/pdb/select?group=true&group.field=pdb_id&group.ngroups=true&json.nl=map&start=0&sort=fasta(e_value) asc&xjoin_fasta=true&bf=fasta(percentIdentity)&xjoin_fasta.external.expupperlim=0.1&xjoin_fasta.external.sequence=
MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELNH
PNIVKLLDVIHTENKLYLVFEFLHQDLKKFMDASALTGIPLPLIKSYLFQLLQGLAFCHS
HRVLHRDLKPQNLLINTEGAIKLADFGLARAFGVPVRTYTHEVVTLWYRAPEILLGCKYY
STAVDIWSLGCIFAEMVTRRALFPGDSEIDQLFRIFRTLGTPDEVVWPGVTSMPDYKPSF
PKWARQDFSKVVPPLDEDGRSLLSQMLHYDPNKRISAKAALAHPFFQDVTKPVPHLRL&q=*:*&fq={!xjoin}xjoin_fasta&fl=pfam_accession,pdb_id,molecule_name,ec_number,uniprot_accession_best,tax_id&wt=json&rows=10
Number of results 10


Print the first result to see what we have

In [4]:
pprint(first_results[0])

{'e_value': 6.7e-76,
 'ec_number': ['2.7.11.22'],
 'molecule_name': ['Cyclin-dependent kinase 2'],
 'pdb_id': '3ezr',
 'percent_identity': 100.0,
 'pfam_accession': ['PF00069'],
 'tax_id': [9606],
 'uniprot_accession_best': ['P24941']}
